In [1]:
import pandas as pd
import re
df_t = pd.read_csv('transcript_all.csv')
df_p = pd.read_csv('product_all.csv')

In [2]:
def findnum(s):
    try:
        return re.findall(r'\d+', str(s))[0]
    except:
        return ''

df_t['tour_group_id']  = df_t.chat_start_url.apply(findnum)

tourid = df_p.loc[df_p.language_code == 'EN',['tour_group_id','name']]
tourid['tour_group_id'] = tourid.tour_group_id.apply(str)
df_m = pd.merge(df_t, tourid, on='tour_group_id')
df_text = df_m.groupby(['chat_id','name']).text.apply(' '.join).reset_index()

In [6]:
df_text.name.value_counts()[:100].index

Index(['Tickets to Colosseum, Roman Forum and Palatine Hill',
       'The Phantom of the Opera', 'La Perle by Dragone',
       'Dubai Aquarium & Underwater Zoo + Burj Khalifa Combo',
       'Fast Track Tickets to Sagrada Familia',
       'Guided Tour of the Vatican Museums, Sistine Chapel and St. Peter's Basilica',
       'Bateaux Parisiens Seine River Sightseeing Cruise',
       'Skip the Line Tickets to the Vatican Museums and Sistine Chapel',
       'Aladdin',
       'Red Dune Desert Safari with BBQ Dinner & Live Entertainment',
       'Eiffel Tower Summit Skip The Line Tickets with Optional Seine River Cruise',
       'Burj Khalifa: At the Top (Level 124 & 125)',
       'City of Lights: Paris After Dark Sightseeing Cruise',
       'IMG Worlds of Adventure',
       'Escorted Exclusive Skip The Line Tickets to Vatican Museum & Sistine Chapel',
       'Skip The Line: Eiffel Tower Tickets with Host', 'The Lion King',
       'Sagrada Familia Fast Track Tickets with Tower Access and Audi

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=20)
t = tv.fit_transform(df_text.name.value_counts()[:100].index)
terms = pd.DataFrame({'term': tv.get_feature_names(), 'tfidf': t.toarray().sum(0)})
t.shape

(100, 515)

In [25]:
x = tv.transform(df_text.text)

In [26]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df_text.name)

In [27]:
y.shape

(4299,)

In [29]:
from sklearn.ensemble import RandomForestClassifier

/home/tianyi/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [58]:
%%time
rf_params = {
    'n_jobs': -1,
    'n_estimators': 40,
    'max_depth': 60,
    'random_state': 42
}
rf = RandomForestClassifier(**rf_params)

rf.fit(x, y)

CPU times: user 1.93 s, sys: 19.4 ms, total: 1.95 s
Wall time: 634 ms


In [32]:
from sklearn.metrics import accuracy_score

In [59]:
%%time
accuracy_score(rf.predict(x), y)

CPU times: user 1.45 s, sys: 754 ms, total: 2.2 s
Wall time: 619 ms


0.6247964642940219

In [33]:
%%time
accuracy_score(rf.predict(x), y)

CPU times: user 19.2 s, sys: 8.36 s, total: 27.6 s
Wall time: 9.52 s


0.711560828099558

In [66]:
import joblib
# save label encoder
filename = 'Product_Encoder.sav'
joblib.dump(le, filename)

# save tfidf
filename = 'Product_Vectorizer.sav'
joblib.dump(tv, filename)

# save rf model
filename = 'Product_Model.sav'
joblib.dump(rf, filename)

['Product_Model.sav']

In [67]:
clf = joblib.load('Product_Model.sav')
le1 = joblib.load('Product_Encoder.sav')
tv1 = joblib.load('Product_Vectorizer.sav')

In [71]:
le.inverse_transform([rf.predict(tv.transform(['can i choose seat for wicked']))])[0][0]

/home/tianyi/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


'Wicked'

In [61]:
le.inverse_transform([rf.predict(tv.transform(['what about wicked']))])

/home/tianyi/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([['Wicked']], dtype=object)